In [ ]:
# 1️⃣ Import Required Libraries
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt
import time
import os
from google.colab import drive, files
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# 3️⃣ Upload Torrent File
print("Please upload a .torrent file:")
uploaded = files.upload()
torrent_path = list(uploaded.keys())[0]

# 4️⃣ List Content in Torrent
def list_torrent_files(torrent_path):
    """Lists all files inside a given torrent file."""
    ses = lt.session()
    info = lt.torrent_info(torrent_path)

    print("\nFiles in Torrent:")
    for idx, file in enumerate(info.files()):
        print(f"{idx}: {file.path} ({file.size} bytes)")

    return info

info = list_torrent_files(torrent_path)

# 5️⃣ Selective Download Function
def download_torrent(torrent_path, save_path, file_indices):
    """Downloads only selected files from the torrent."""
    ses = lt.session()
    info = lt.torrent_info(torrent_path)

    params = {
        'ti': info,
        'save_path': save_path,
        'storage_mode': lt.storage_mode_t(2)  # Sparse mode
    }
    torrent_handle = ses.add_torrent(params)

    # Apply selective download settings
    for idx, file in enumerate(info.files()):
        priority = 7 if idx in file_indices else 0  # 7 = high priority, 0 = skip
        torrent_handle.file_priority(idx, priority)

    print("\nDownloading selected files...")

    # UI Progress Bar Setup
    state_str = [
        "queued", "checking", "downloading metadata",
        "downloading", "finished", "seeding",
        "allocating", "checking fastresume"
    ]
    download_bar = widgets.FloatSlider(
        step=0.01, disabled=True, layout=widgets.Layout(width="100%"),
        style={"description_width": "initial"}
    )
    display(download_bar)

    # Monitor download progress
    while not torrent_handle.is_seed():
        s = torrent_handle.status()

        download_bar.description = f"{torrent_handle.name()} | {s.download_rate / 1000:.2f} kB/s | {state_str[s.state]}"
        download_bar.value = s.progress * 100

        time.sleep(1)

    print("\nDownload Complete!")

# 6️⃣ Ask User for Selective Download
save_directory = "/content/drive/MyDrive/Torrents"

num_files = len(info.files())
selected_files = input(f"Enter file indices to download (comma-separated, 0-{num_files - 1}): ")
selected_files = [int(idx.strip()) for idx in selected_files.split(",")]

download_torrent(torrent_path, save_directory, file_indices=selected_files)
